# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [21]:
filepath = '../data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [22]:
filepath = '../data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [6]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [7]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [8]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [9]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


## Матрица рейтингов

In [10]:
from scipy.sparse import coo_matrix, csr_matrix

In [11]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [ ]:
R

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [12]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [ ]:
user_1 = R[0]
user_1

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [ ]:
user_2 = R[1]
user_2

Мы можем сравнивать элементы с 0

In [ ]:
user_1_rated = (user_1 != 0)
user_1_rated

Можем их "индексировать"

In [ ]:
user_1[user_1_rated]

Можем считать количество ненулевых элементов

In [ ]:
user_1.nnz

Можем умножать 2 разреженных вектора поэлементно:

In [ ]:
(user_1_rated).multiply(user_2_rated)

и скалярно

In [ ]:
user_1.dot(user_2.T)[0, 0]

И превращать разреженную матрицу (вектор) в плотную

In [ ]:
user_1_dense = user_1.toarray()
user_1_dense

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


In [13]:
from scipy.spatial.distance import cosine
# Используем библиотечную функцию из лекции

def cosine_similarity_pair_users(u, v):
    idx = (u != 0).multiply(v != 0)
    return 0 if idx.nnz < 3 else (1 - cosine(u[idx], v[idx]))


### _Решение_

In [15]:
cosine_similarity_pair_users(R[19], R[716])
# 0.9605870152896533

0.9605870152896533

## Функция прогнозирования рейтинга

Реализуйте функцию, которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей (несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)

и возвращает вектор с предсказанными рейтингами по всем товарам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [17]:
def rate_items_user(u, R, n_neigbours=30):
    # Косинусное растояние между пользователями
    df_dist = pd.DataFrame(data=[[i, cosine_similarity_pair_users(R[u], R[i])]
                                 for i in range(R.shape[0])], 
                           columns=['id', 'cosine'])
    # Исключаем искомого пользователя
    df_dist.drop([u], inplace=True)

    # Сортируем список по опхожести и отбираем 30 ближайших
    df_users = df_dist.sort_values(by='cosine', ascending=False)[0:n_neigbours]
    
    # Вычисляем знаметатель формулы рейтинга
    abs_sum = df_users.cosine.abs().sum()
    # считаем рейтинг
    rating = pd.DataFrame(data=[(df_users.cosine.values * R[df_users.id, i]) / abs_sum 
                                for i in range(0, R.shape[1])],
                         columns=['rating'])
    return rating


В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).
* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

### _Решение_

In [18]:
# ID пользователя
usr_id = 20

# Получаем список фильмов с предсказанным рейтингом
df_rating = rate_items_user(usr_id, R)
# Исключаем просмотренное
df_rating.drop(R[usr_id].nonzero()[1], inplace=True)
# Сортируем
df_rating = df_rating.rating.sort_values(ascending=False)

# Топ-5
top5 = df_rating[:5].index

## Генерация ответа

In [20]:
', '.join(str(i) for i in top5)
# 2614, 306, 343, 5573, 6720

'2614, 306, 343, 5573, 6720'